In [ ]:
import pandas as pd 
import sys
import os 
from dotenv import load_dotenv


sys.path.append('C:\\Users\\Ourba\\Desktop\\10Academy\\10ac_week1_telecom_analysis\\src')
from loader import load_data


load_dotenv()

user = os.getenv('PG_USER')
password = os.getenv('PG_PASSWORD')
database_name = 'telecom'
table_name= 'xdr_data'
host = 'localhost'
port = '5432'

#Load data from Loader Module
df = load_data(host, port, user, password, database_name, table_name)
print(df.head())

: 

In [ ]:
from scipy import stats
import numpy as np
from utils import preproccess_numerical_data
preproccess_numerical_data(df)

numerical_cols = df.select_dtypes(include=['number']).columns

z_scores = stats.zscore(df[numerical_cols])
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
df = df[filtered_entries]
print(df[filtered_entries])

: 

In [ ]:
import pandas as pd

# Calculate session frequency per customer (MSISDN)
session_frequency = df.groupby('MSISDN/Number').size().reset_index(name='Session Frequency')

# Calculate session duration per customer (MSISDN) in milliseconds
session_duration = df.groupby('MSISDN/Number')['Dur. (ms)'].sum().reset_index(name='Session Duration (ms)')

# Calculate total session traffic (download + upload) per customer (MSISDN) in bytes
session_traffic = df.groupby('MSISDN/Number')[['Total UL (Bytes)', 'Total DL (Bytes)']].sum().sum(axis=1).reset_index(name='Total Session Traffic (Bytes)')

# Merge the aggregated metrics into a single DataFrame
engagement_metrics = pd.merge(session_frequency, session_duration, on='MSISDN/Number')
engagement_metrics = pd.merge(engagement_metrics, session_traffic, on='MSISDN/Number')

# Rank the customers based on each engagement metric
engagement_metrics['Rank - Session Frequency'] = engagement_metrics['Session Frequency'].rank(ascending=False)
engagement_metrics['Rank - Session Duration'] = engagement_metrics['Session Duration (ms)'].rank(ascending=False)
engagement_metrics['Rank - Total Session Traffic'] = engagement_metrics['Total Session Traffic (Bytes)'].rank(ascending=False)

# Report the top 10 customers for each engagement metric
top_10_frequency = engagement_metrics.sort_values(by='Rank - Session Frequency').head(10)
top_10_duration = engagement_metrics.sort_values(by='Rank - Session Duration').head(10)
top_10_traffic = engagement_metrics.sort_values(by='Rank - Total Session Traffic').head(10)

print("Top 10 Customers by Session Frequency:")
print(top_10_frequency[['MSISDN/Number', 'Session Frequency']])

print("\nTop 10 Customers by Session Duration:")
print(top_10_duration[['MSISDN/Number', 'Session Duration (ms)']])

print("\nTop 10 Customers by Total Session Traffic:")
print(top_10_traffic[['MSISDN/Number', 'Total Session Traffic (Bytes)']])


: 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


# Assuming engagement_metrics is your DataFrame containing the engagement metrics
# Normalize the engagement metrics
scaler = StandardScaler()
normalized_metrics = scaler.fit_transform(engagement_metrics[['Session Frequency', 'Session Duration (ms)', 'Total Session Traffic (Bytes)']])

# Run k-means clustering with k=3
kmeans = KMeans(n_clusters=3, random_state=42)
engagement_metrics['Cluster'] = kmeans.fit_predict(normalized_metrics)

# Analyze and interpret the clusters
cluster_summary = engagement_metrics.groupby('Cluster').mean().reset_index()

print("Cluster Summary:")
print(cluster_summary)

# Report the top 10 customers for each cluster
for cluster_id in range(3):
    print(f"\nTop 10 Customers in Cluster {cluster_id}:")
    print(engagement_metrics[engagement_metrics['Cluster'] == cluster_id].sort_values(by='Rank - Session Frequency').head(10)[['MSISDN/Number', 'Session Frequency']])


: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate minimum, maximum, average, and total non-normalized metrics for each cluster
cluster_summary = engagement_metrics.groupby('Cluster').agg({
    'Session Frequency': ['min', 'max', 'mean', 'sum'],
    'Session Duration (ms)': ['min', 'max', 'mean', 'sum'],
    'Total Session Traffic (Bytes)': ['min', 'max', 'mean', 'sum']
}).reset_index()

# Rename columns for better readability
cluster_summary.columns = ['Cluster', 
                           'Min Session Frequency', 'Max Session Frequency', 'Avg Session Frequency', 'Total Session Frequency',
                           'Min Session Duration', 'Max Session Duration', 'Avg Session Duration', 'Total Session Duration',
                           'Min Total Session Traffic', 'Max Total Session Traffic', 'Avg Total Session Traffic', 'Total Total Session Traffic']

print("Cluster Summary:")
print(cluster_summary)

# Visualize the results
metrics = ['Session Frequency', 'Session Duration (ms)', 'Total Session Traffic (Bytes)']

# Reshape the data for plotting
plot_data = pd.melt(cluster_summary, id_vars=['Cluster'], var_name='Metric', value_name='Value')

# Plot grouped bar charts
plt.figure(figsize=(12, 8))
sns.barplot(x='Metric', y='Value', hue='Cluster', data=plot_data, ci=None)
plt.title('Comparison of Engagement Metrics Across Clusters')
plt.xlabel('Engagement Metric')
plt.ylabel('Value')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Cluster', loc='upper right')
plt.show()


: 

Cluster 0: This cluster seems to represent users with low to moderate engagement levels. The session frequency, duration, and total session traffic are relatively low compared to Cluster 2 but higher than Cluster 1.
Cluster 1: This cluster seems to represent users with unusually high and identical engagement levels across all metrics, which is suspicious and may indicate an issue with the clustering or aggregation process.
Cluster 2: This cluster seems to represent users with low engagement levels. The session frequency, duration, and total session traffic are the lowest among the three clusters.

In [ ]:
import pandas as pd

# List of individual application columns
app_cols = ['Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']

# Create new columns for total traffic per application
for app in app_cols:
    df[f'{app} Total Traffic (Bytes)'] = df[f'{app} DL (Bytes)'] + df[f'{app} UL (Bytes)']

# Aggregate total traffic per application for each user
user_traffic = df.groupby('MSISDN/Number')[[f'{app} Total Traffic (Bytes)' for app in app_cols]].sum().reset_index()

# Initialize an empty dictionary to store the top 10 most engaged users per application
top_10_users_per_app = {}

# Identify the top 10 most engaged users per application
for app in app_cols:
    top_10_users_per_app[app] = user_traffic.nlargest(10, f'{app} Total Traffic (Bytes)')[['MSISDN/Number', f'{app} Total Traffic (Bytes)']]

# Display the top 10 most engaged users per application
for app in app_cols[1:]:  # Start from Email and proceed to Other
    print(f"Top 10 Most Engaged Users for {app}:")
    print(top_10_users_per_app[app].head(10))
    

: 

In [ ]:



# Calculate the total traffic for each application
total_traffic_per_app = {app: df[f'{app} Total Traffic (Bytes)'].sum() for app in app_cols}

# Sort the applications based on total traffic in descending order
sorted_apps = sorted(total_traffic_per_app, key=total_traffic_per_app.get, reverse=True)

# Select the top 3 most used applications
top_3_apps = sorted_apps[:3]

# Create a bar plot for the top 3 most used applications
plt.figure(figsize=(10, 6))
sns.barplot(x=top_3_apps, y=[total_traffic_per_app[app] for app in top_3_apps])
plt.xlabel('Applications')
plt.ylabel('Total Traffic (Bytes)')
plt.title('Top 3 Most Used Applications by Total Traffic')
plt.show()


: 

In [ ]:
from sklearn.cluster import KMeans

# Select the engagement metrics for clustering
engagement_metrics = df[['Social Media Total Traffic (Bytes)', 'Google Total Traffic (Bytes)', 'Email Total Traffic (Bytes)']]

# Calculate WCSS for different values of k
wcss = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(engagement_metrics)
    wcss.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS')
plt.xticks(np.arange(1, 11, 1))
plt.grid(True)
plt.show()


: 